# **Códigos utilizados nos dados**

Iniciar importando os pacotes

In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import os
import requests
from dbfread import DBF
import psycopg2
from sqlalchemy import create_engine

## Leitor auxilair para arquivos .dbc

Caso o arquivo esteja no formato .dbc, ao invés de carregar com pd.read_csv(), utilize:

`dbf_to_dataframe(Caminho da pasta/nome_do_arquivo.dbf)`

Essa funçao carrega o arquivo .dbc e converte em um pandas dataframe,  <br>
assim será mais simples de manipular e converter para outros formatos

## *Código*

In [ ]:
# Criar uma variável para substituir `pasta` com o caminho do arquivo
def dbf_to_dataframe(pasta):
    # Lista vazia
    dados = []
    # Ler o DBF e fazer append na lista vazia
    for dado in DBF(pasta, encoding='latin1'):
        dados.append(dado)

    # Converter em dadosframe
    return pd.DataFrame(dados)


## Loop para concatenar os dataframes



1 - O código criaa um DataFrame vazio chamado `concatenated_df` que será usado para armazenar os dados de todos os arquivos CSV concatenados.
<br>
<br>
2 - É definida a variável `folder_path`, que representa o caminho do diretório onde os arquivos CSV estão localizados.
<br>
<br>
3 - É definida uma função chamada `anything(df)`, que recebe um DataFrame df como entrada e retorna o mesmo DataFrame sem fazer qualquer alteração nele.
<br>
<br>
4 - O código entra em um `loop for` para percorrer cada arquivo no diretório `folder_path`
<br>
<br>
5 - É verificado se o nome do arquivo começa com "SIM" e termina com a extensão ".csv". Se sim, o arquivo é processado
<br>
<br>
6 - O caminho completo do arquivo é construído usando `os.path.join()` com base no caminho do diretório `folder_path` e o nome do arquivo atual (`filename`).
<br>
<br>
7 - `pd.read_csv()` é usado para ler o arquivo CSV em um DataFrame chamado df. Os parâmetros `sep=';'` e `encoding='latin1'` especificam que o separador no arquivo CSV é ponto e vírgula e a codificação é 'latin1'.
<br>
<br>
8 - O DataFrame df é passado para a função `nada()` definida anteriormente. Essa função não faz nenhuma alteração no DataFrame, então o DataFrame tratado (`treated_df`) será idêntico ao DataFrame original.
<br>
<br>
9 - O DataFrame tratado (`treated_df`) é então concatenado com o DataFrame `concatenated_df` usando `pd.concat()`. A concatenação é feita ao longo das linhas, o que significa que os dados dos arquivos CSV são empilhados verticalmente no DataFrame `concatenated_df`.
<br>
<br>
10 - Após o loop ser concluído e todos os arquivos CSV serem processados e concatenados, o índice do DataFrame `concatenated_df` é redefinido usando `reset_index(drop=True, inplace=True)`. Isso garante que o índice seja redefinido sequencialmente de 0 até o número total de linhas no DataFrame concatenado, e o parâmetro `drop=True` indica que o índice anterior será descartado.
<br>
<br>
11 - O código `display()` exibe o novo DataFrame

## *Códigos*
1 - Concatenar .csv

In [ ]:
#1 - DataFrame vazio para armazenar os dados concatenados
concatenated_df = pd.DataFrame()

#2 - Pasta onde os dados estão armazenados
folder_path = 'dado_tratado'

#3 - Funçao
def anything(df):
    return df

#4 - Loop nos arquivos no diretório
for filename in os.listdir(folder_path):
    if filename.startswith(f'SIM') and filename.endswith('.csv'):  #5 - Construção do nome do arquivo
        file_path = os.path.join(folder_path, filename)            #6 - Construção do caminho completo
        df = pd.read_csv(file_path,sep = ';', encoding='latin1')   #7 - Leitura do arquivo

        # Selecionar colunas
        #df = df[['']]
        treated_df = anything(df)                                  #8 - Aplica a função que retorna o mesmo df
        concatenated_df = pd.concat([concatenated_df, treated_df]) #9 - Concatena os df

#10 - Reset index e concatenando em um DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

#11 - Display the final concatenated DataFrame
display(concatenated_df)

2 - Concatenar DBF

In [ ]:
#1 - DataFrame vazio para armazenar os dados concatenados
concatenated_df = pd.DataFrame()

#2 - Pasta onde os dados estão armazenados
folder_path = 'C:\\vitalstrategies\\data_sicence\\VIOLBR18.dbf'

#3 - Funçao
def anything(df):
    return df

#4 - Loop nos arquivos no diretório
for filename in os.listdir(folder_path):
    if filename.startswith(f'VIOLBR') and filename.endswith('.dbf'):  #5 - Construção do nome do arquivo
        file_path = os.path.join(folder_path, filename)               #6 - Construção do caminho completo
        df = dbf_to_dataframe(file_path)                              #7 - Leitura do arquivo

        # Selecionar colunas
        df = df[[
            'ID_AGRAVO',
            'DT_NOTIFIC',
            'NU_ANO'
            ]]

        treated_df = anything(df)                                     #8 - Aplica a função que retorna o mesmo df
        concatenated_df = pd.concat([concatenated_df, treated_df])    #9 - Concatena os df

#10 - Reset index e concatenando em um DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

#11 - Display the final concatenated DataFrame
display(concatenated_df)

## Salvar o DataFrame direto no banco Postgres
Exemplo de como inputar um dataframe no postgres

In [ ]:
# Carregar os dados
df = pd.read_csv('C:/vitalstrategies/data_sicence/ISM/Dados/ipea/ipea_indicadores/ivs_ipea.csv', sep = ';')
df.head()

exemplo de alterações possíveis

In [ ]:
# Seleção das variáveis
df = df[['Município', 'Ano', 'IVS', 'IVS Infraestrutura Urbana', 'IVS Capital Humano', 'IVS Renda e Trabalho', 'UF']]

# Renomeando as variáveis
df = df.rename(columns={
    'Município': 'ibge_muni',
    'Ano' : 'id_ano',
    'IVS': 'ivs',
    'IVS Infraestrutura Urbana': 'ivs_infra_urb',
    'IVS Capital Humano': 'ivs_cap_hum',
    'IVS Renda e Trabalho': 'ivs_rend_trab',
    'UF': 'uf'
})


In [ ]:
colunas = ['UF', 'Município', 'Ano', 'IVS']
df = df[colunas]

In [ ]:
# Configurações do banco de dados
db_config = {
    'host': 'ls-5ae56aeb2ca5d6cbcd113e0c1aaa14c2a5395de9.c1ozwzg2gdjm.us-east-1.rds.amazonaws.com',
    'database': 'postgres',
    'user': 'postgresql_vs_public',
    'password': '>D,&g-5ZURExX=e-t]8?UXL2j!9B;xtc',
}

###################################################################
## Nome da tabela que será inputada/atualizada no banco de dados ##
###################################################################
nome_tabela = 'tb_pop_ms'



# Código que tenta conectar ao banco e carregar o dataframe
try:
    # Conecta ao banco de dados usando o SQLAlchemy
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

    # Carrega o DataFrame para o banco de dados
    concatenated_df.to_sql(dataframe_2021, engine, if_exists='append', index=False) # alterar "replace" no if_exists para "append" e testar com as novas variaveis

    # Caso dê certo
    print("Dados carregados com sucesso!")

    # Caso não dê certo
except (psycopg2.Error, Exception) as e:
    print("Erro ao conectar ao PostgreSQL ou carregar os dados:", e)

Dados carregados com sucesso!


In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import os
import requests
from dbfread import DBF
import psycopg2
from sqlalchemy import create_engine


###################################################################
## Nome da tabela que será inputada/atualizada no banco de dados ##
###################################################################

# Nome que da tabela para o SQL
nome_tabela = 'tb_sinan_viol'

# Pasta onde os dados estão armazenados
folder_path = 'C:/Users/João Miguel/OneDrive/Documentos/Python Scripts/dado_convertido_dbf'
# Iniciais do arquivo
arquivo = 'VIOLBR1'

# Seleção das variáveis
variaveis_selecionadas = ['ID_MUNICIP',
                        'TP_NOT',
                         'ID_AGRAVO',
                         'DT_NOTIFIC',
                         'DT_OCOR',
                         #'DT_NASC',
                         'ANO_NASC',
                         'NU_IDADE_N',
                         'CS_SEXO',
                         'CS_RACA',
                         'ID_MN_RESI',
                         'LES_AUTOP']


def tratamento(df):
    df['ID_MUNICIP'] = df['ID_MUNICIP'].astype('int')
    df['TP_NOT'] = df['TP_NOT'].astype('int')
    df['ANO_NASC'] = df['ANO_NASC'].replace('','1800').astype('int')
    df['NU_IDADE_N'] = df['NU_IDADE_N'].replace(np.nan,999).astype('int')
    df['CS_RACA'] = df['CS_RACA'].replace('','999').astype(int)
    df['ID_MN_RESI'] = df['ID_MN_RESI'].replace('','999').astype('int')
    df['LES_AUTOP'] = df['LES_AUTOP'].replace('','9').astype('int')
    return df


###################################################################
## Código para carregar, selecionar variáveis e inputar no Banco ##
###################################################################


# Criar uma variável para substituir `pasta` com o caminho do arquivo
def dbf_to_dataframe(pasta):
    # Lista vazia
    dados = []
    # Ler o DBF e fazer append na lista vazia
    for dado in DBF(pasta, encoding='latin1'):
        dados.append(dado)

    # Converter em dadosframe
    return pd.DataFrame(dados)


# Função para ler o arquivo com base na extensão
def pandas_read(file_path):
    _, file_extension = os.path.splitext(file_path)

    if file_extension == '.csv':
        return pd.read_csv(file_path, sep=';', encoding='latin1')
    elif file_extension == '.dbf':
        return dbf_to_dataframe(file_path)  # Substitua a função dbf_to_dataframe pela função correta para ler arquivos .dbf
    else:
        raise ValueError(f"Tipo de arquivo não suportado: {file_extension}")

# DataFrame vazio para armazenar os dados concatenados
concatenated_df = pd.DataFrame()

# Loop nos arquivos no diretório
for filename in os.listdir(folder_path):
    if filename.startswith(arquivo) and (filename.endswith('.csv') or filename.endswith('.dbf')):
        file_path = os.path.join(folder_path, filename)             # Construção do caminho completo
        df = pandas_read(file_path)                                 # Leitura do arquivo

        # Lista de colunas selecionadas (adicione as colunas desejadas aqui)
        df = df[variaveis_selecionadas]

        df = tratamento(df)

        treated_df = df.copy()                                      # Aplica a função que retorna o mesmo df
        concatenated_df = pd.concat([concatenated_df, treated_df])

        print(f"Arquivo {filename} carregado e concatenado.")
# Reset index e concatenando em um DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

# Display do DataFrame concatenado
display(concatenated_df)


Arquivo VIOLBR10.dbf carregado e concatenado.
Arquivo VIOLBR11.dbf carregado e concatenado.
Arquivo VIOLBR12.dbf carregado e concatenado.
Arquivo VIOLBR13.dbf carregado e concatenado.
Arquivo VIOLBR14.dbf carregado e concatenado.
Arquivo VIOLBR15.dbf carregado e concatenado.
Arquivo VIOLBR16.dbf carregado e concatenado.
Arquivo VIOLBR17.dbf carregado e concatenado.
Arquivo VIOLBR18.dbf carregado e concatenado.
Arquivo VIOLBR19.dbf carregado e concatenado.


,ID_MUNICIP,TP_NOT,ID_AGRAVO,DT_NOTIFIC,DT_OCOR,ANO_NASC,NU_IDADE_N,CS_SEXO,CS_RACA,ID_MN_RESI,LES_AUTOP
0,500370,2,Y09,2010-01-10,2010-01-10,1997,4012,F,9,500370,1
1,221130,2,Y09,2010-02-02,2010-02-02,1994,4015,M,2,221130,2
2,230526,2,Y09,2010-01-01,2010-01-01,1994,4015,F,4,230526,2
3,292660,2,Y09,2010-01-08,2010-01-08,1973,4036,F,2,292660,2
4,251398,2,Y09,2010-01-15,2010-01-15,1962,4047,F,4,251398,2
...,...,...,...,...,...,...,...,...,...,...,...
2259571,310670,2,Y09,2019-03-21,2019-03-07,1994,4024,F,9,310670,2
2259572,310670,2,Y09,2019-02-01,2018-07-11,1957,4061,F,4,310670,2
2259573,310670,2,Y09,2019-03-06,2019-02-27,1986,4032,F,9,310670,1
2259574,310670,2,Y09,2019-03-06,2019-03-01,1990,4028,F,9,310670,1


In [ ]:
#####################################
## Configurações do banco de dados ##
#####################################

db_config = {
    'host': 'ls-5ae56aeb2ca5d6cbcd113e0c1aaa14c2a5395de9.c1ozwzg2gdjm.us-east-1.rds.amazonaws.com',
    'database': 'postgres',
    'user': 'postgresql_vs_public',
    'password': '>D,&g-5ZURExX=e-t]8?UXL2j!9B;xtc',
}

# Código que tenta conectar ao banco e carregar o dataframe
try:
    # Conecta ao banco de dados usando o SQLAlchemy
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

    # Carrega o DataFrame para o banco de dados
    concatenated_df.to_sql(nome_tabela, engine, if_exists='replace', index=False) # alterar "replace" no if_exists para "append" e testar com as novas variaveis

    # Caso dê certo
    print("Dados carregados com sucesso!")

    # Caso não dê certo
except (psycopg2.Error, Exception) as e:
    print("Erro ao conectar ao PostgreSQL ou carregar os dados:", e)

Dados carregados com sucesso!


In [ ]:
folder_path = ''
# Iniciais do arquivo
arquivo = ''

a =dbf_to_dataframe('C:\\vitalstrategies\\data_sicence\\VIOLBR18.dbf')